# Title
[]()

In [1]:
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
from silvhua import *
import pickle
import os
import requests
import json
from pandas import json_normalize  

In [7]:
sys.path.append(r'C:\ProgramData\Anaconda3\Scripts')
sys.path.append(r'C:\ProgramData\Anaconda3')
sys.path.append(r'C:\ProgramData\Anaconda3\Library\bin')

In [12]:
with open("credentials.json") as f:
    credentials = json.load(f)
# user_id = os.environ['fb_user_id']

In [ ]:
def get_user_ig_post(ig_user_id, access_token, pages=5, filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    user_id = str(ig_user_id)
    url_root = "https://graph.facebook.com/v15.0/"
    # url_without_token = f'{url_root}{ig_user_id}/media?fields=caption%2Ccomments_count%2Cis_shared_to_feed%2Clike_count%2Cmedia_type%2Cmedia_url%2Cmedia_product_type%2Cpermalink%2Ctimestamp%2Ccomments&access_token='
    # url_without_token = f'{url_root}{ig_user_id}/media?fields=caption,comments_count,is_shared_to_feed,like_count,media_type,media_url,media_product_type,permalink,timestamp,comments&access_token='
    url_without_token = f'{url_root}{ig_user_id}/media?access_token='
    url = url_without_token+access_token
    print(url_without_token)
    response_json_dict = dict()
    df_list = []
    for page in range(1,pages+1):
        response = requests.get(url)
        print('Response status code: ',response.status_code)
        response_json_dict[page] = response.json()
        df_list.append(json_normalize(response_json_dict[page], record_path='data'))
        try:
            url = response_json_dict[page]['paging']['next']
        except: 
            pass
    df = pd.concat(df_list)
    print('Number of posts:',len(df))
    if filename:
        try:
            save_csv(df,filename,csv_path)
            savepickle(response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

df_test, response_json_test = get_user_ig_post(credentials['ig_user_id'], 
        access_token, pages=2, filename='my_ig_posts_2p_2023-01-22')
"""
SH 2023-01-22 23:14
SSLError: Can't connect to HTTPS URL because the SSL module is not available.

During handling of the above exception, another exception occurred:

MaxRetryError                             Traceback (most recent call last)
File ~\AppData\Roaming\Python\Python39\site-packages\requests\adapters.py:489, in HTTPAdapter.send(self, request, stream, timeout, verify, cert, proxies)
    488 if not chunked:
...
--> 563         raise SSLError(e, request=request)
    565     raise ConnectionError(e, request=request)
    567 except ClosedPoolError as e:

SSLError: HTTPSConnectionPool(host='graph.facebook.com', port=443): Max retries exceeded with url: /v15.0/17841403231458630?access_token=EAAKEofVXnvEBAFkvIH79iHq39i7tiYOe5LBMvO6kr7pq4iHleW1d763MxoYi2ZANGb3pHm1jwdFZBy2IYsMdBZAV45L3RCnjqjsdVEZA2ui8gkswqWaSZCfBFLeaK0muSbWOGo2Lf6Vu24XaHLZAQVtpfToXT1auPo6s65LOEpMceS6URnnxcDKLEnKdvnKvbCqkVwDEJo57wDxEk80q9JknNcZBKnMzhGEHaNXn7SVeFt9BHdOMfqtPCC8kky2VHwZD (Caused by SSLError("Can't connect to HTTPS URL because the SSL module is not available."))
"""

# *End of Page*